In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
import ipywidgets as widgets

In [264]:
# Rosenbrock function
def rosen(x):
    return 100*(x[1]-x[0]**2)**2+(1-x[0])**2

def rosen_der(x):
    der = np.zeros(2)
    der[0] = -400*x[0]*(x[1]-x[0]**2)-2*(1-x[0])
    der[1] = 200*(x[1]-x[0]**2)
    return der

def rosen_hess(x):
    hess = np.array([[1200*x[0]**2-400*x[1]+2, -400*x[0]],
                     [-400*x[0], 200]])
    return hess

In [286]:
# Gradient Descent
class GradientDescent:
    def __init__(self, grad, eps=1e-4, tol=1e-6, n=1e6):
        self.eps = eps
        self.tol = tol
        self.grad = grad
        self.n = int(n)
        
    def __Next(self, x):
        xp = x - self.eps*self.grad(x)
        return xp
    
    def __Norm(self, x):
        return np.sqrt(np.sum(self.grad(x)**2))
    
    def Optim(self, p):
        result = []
        result.append(p)
        for _ in range(self.n):
            p = self.__Next(p)
            result.append(p)
            if self.__Norm(p) < self.tol:
                break
        result = np.asarray(result)
        return result

In [279]:
# Newton's Method
class Newton:
    def __init__(self, grad, hess, eps=1e-4, tol=1e-6, n=1e6):
        self.eps = eps
        self.tol = tol
        self.grad = grad
        self.hess = hess
        self.n = int(n)
        
    def __Next(self, x):
        xp = x - self.eps*np.matmul(np.linalg.inv(self.hess(x)),self.grad(x))
        return xp
    
    def __Norm(self, x):
        return np.sqrt(np.sum(self.grad(x)**2))
    
    def Optim(self, p):
        result = []
        result.append(p)
        for _ in range(self.n):
            p = self.__Next(p)
            result.append(p)
            if self.__Norm(p) < self.tol:
                break
        result = np.asarray(result)
        return result

In [ ]:
# Quasi-Newton Method
class QuasiNewton:
    def __init__(self, grad, eps=1e-4, tol=1e-6, n=1e5, method="SR1"):
        self.eps = eps
        self.tol = tol
        self.grad = grad
        self.n = int(n)
        self.H = np.identity(2)
        self.method = method
        
    def __Next(self, x):
        xp = x - self.eps*np.matmul(self.H, self.grad(x))
        s = xp - x
        y = self.grad(xp) - self.grad(x)
        if self.method=="SR1":
            self.H = self.H + np.outer(s - np.matmul(self.H,y), s - np.matmul(self.H,y))/np.dot(s - np.matmul(self.H,y), y)
        elif self.method=="DFP":
            self.H = self.H - np.matmul(np.outer(np.matmul(self.H, y), y), self.H)/np.dot(y, np.matmul(self.H, y)) + np.outer(s,s)/np.dot(y,s)
        elif self.method=="BFGS":
            self.H = np.matmul(np.matmul(np.identity(2)-np.outer(s,y)/np.dot(y,s), self.H), np.identity(2)-np.outer(y,s)/np.dot(y,s)) + np.outer(s,s)/np.dot(y,s)
        return xp
    
    def __Norm(self, p):
        return np.sqrt(np.sum(self.grad(p)**2))
    
    def Optim(self, p):
        result = []
        result.append(p)
        for _ in range(self.n):
            p = self.__Next(p)
            result.append(p)
            if self.__Norm(p) < self.tol:
                break
        result = np.asarray(result)
        return result

In [292]:
# 최적화
optimizer = QuasiNewton(rosen_der, method="BFGS")
p = np.array([4,3])
result = optimizer.Optim(p)
result

array([[4.        , 3.        ],
       [1.9194    , 3.26      ],
       [1.91993629, 3.26439442],
       ...,
       [1.0036646 , 1.00732171],
       [1.00366423, 1.00732098],
       [1.00366387, 1.00732025]])

In [302]:
# 그림 그리기
w1 = widgets.IntSlider(min=-180, max=180, step=1, value=43, continuous_update=False)
w2 = widgets.IntSlider(min=-180, max=180, step=1, value=120, continuous_update=False)
w3 = widgets.IntSlider(min=1, max=100, step=1, value=10, continuous_update=False)
w4 = widgets.IntSlider(min=1, max=100, step=1, value=10, continuous_update=False)
@interact(a=w1, b=w2, size_x=w3, size_y=w4)
def graph(a, b, size_x, size_y):
    # 함수 
    n = 100
    x = np.linspace(-size_x, size_x, n)
    y = np.linspace(-size_y, size_y, n)
    X, Y = np.meshgrid(x, y)
    Z = rosen([X, Y])
    
    # 최적화 경로
    p_x = result[:,0]
    p_y = result[:,1]
    p_z = rosen([p_x, p_y])

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(1,1,1, projection='3d')
    ax.plot3D(p_x, p_y, p_z, c='red')
    ax.plot_surface(X,Y,Z, rstride=10, cstride=10, cmap='viridis')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Rosenbrock Function')
    ax.view_init(a, b)
    plt.show()

interactive(children=(IntSlider(value=43, continuous_update=False, description='a', max=180, min=-180), IntSli…